# Reto 01 - Prompt Engineering

Consejo rápido: Para ver la Tabla de Contenidos de este Notebook en VS Code, en la pestaña "Explorer", expande la sección "Outline".

## 1. Experimentación de Parámetros
Primero configuraremos el Desafío. Carga la clave API y las bibliotecas de Python relevantes utilizando las celdas a continuación.

### 1.1 Preparación
#### 1.1.1 Configuración del Entorno


#### Crea o actualiza el archivo de entorno llamado ".env"
Hemos incluido un archivo de entorno de ejemplo para tu referencia. Puedes encontrar el archivo `.env.sample` en la raiz de este repositorio o en la carpeta `Student/Resources` dentro de la documentación del desafío. Siéntete libre de hacer las modificaciones necesarias y renombra el archivo como `.env`.

**NOTA:** Asegúrate de mantener el nombre de cada modelo igual al nombre de implementación del modelo que configuraste en Azure OpenAI.

**NOTA:** Ten en cuenta que algunos modelos de AOAI podrían estar obsoletos. Deberías crear los que estén disponibles según sea necesario. Para completar este primer desafío, gpt-35-turbo es el único modelo que necesitas.

Para obtener información sobre los modelos actuales, consulta: [Modelos del Servicio Azure OpenAI Service](https://learn.microsoft.com/es-mx/azure/ai-services/openai/concepts/models)

####  Configuración del Entorno de Azure OpenAI

In [1]:
import openai
import os
import json
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()

assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"

openai.api_base = RESOURCE_ENDPOINT
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

chat_model=os.getenv("CHAT_MODEL_NAME")


#### 1.1.2 Explicación de Parámetros
Especificamos los hiperparámetros para los modelos de Azure OpenAI dentro de las funciones de ayuda. Los usuarios pueden ajustar los parámetros según sus necesidades.


###### **Temperatura**
La temperatura varía de 0 a 2. Aquí tienes un desglose rápido de cómo funciona:
- Temperatura baja (0 a 0.3): Salidas más enfocadas, coherentes y conservadoras.
- Temperatura media (0.3 a 0.7): Creatividad y coherencia equilibradas.
- Alta temperatura (> 0.7): Altamente creativa y diversa, pero potencialmente menos coherente.

###### **Top_p**
Establece el límite de probabilidad para el muestreo de tokens, afectando la amplitud de opciones que considera la IA. Valores más altos conducen a más aleatoriedad, mientras que valores más bajos resultan en salidas más enfocadas.


El parámetro "top_p" actúa como un filtro que controla cuántas palabras o frases diferentes considera el modelo de lenguaje cuando intenta predecir la siguiente palabra. Si configuras el valor de "top p" a 0.5, el modelo de lenguaje solo considerará las 50 palabras o frases más probables que podrían seguir. Pero si configuras el valor de "top p" a 0.9, el modelo de lenguaje considerará las 90 palabras o frases más probables.

Dado que "top_p" y "temperatura" realizan trabajos similares como hiperparámetros, usualmente solo ajustamos uno de ellos en lugar de ambos al mismo tiempo.

###### **Max_tokens**
Max_tokens determina la longitud máxima del texto generado. Al establecer un límite, puedes controlar cuánto texto devolverá el modelo LLM, asegurando que no proporcione una respuesta demasiado larga.

###### **Frequency_penalty**
La penalización de frecuencia asegura que el texto generado sea variado al dar una penalización a los tokens que ya se han utilizado en la respuesta.

Varía de -2.0 a 2.0, con valores más altos resultando en una salida más diversa.

Los valores positivos penalizan nuevos tokens basados en su frecuencia existente en el texto hasta el momento, disminuyendo la probabilidad del modelo de repetir la misma línea textualmente.

#### 1.1.3 Función de Ayuda
A lo largo de este desafío, utilizaremos el modelo `gpt-3.5-turbo` de Azure OpenAI y el [punto final de conexión (endpoint) de chat completions](https://platform.openai.com/docs/guides/chat). 

Esta función de ayuda facilitará el uso de prompts y la visualización de las salidas generadas.

**get_chat_completion** ayuda a crear la respuesta de Azure OpenAI utilizando el modelo de chat de tu elección.

**get_completion_from_messages** ayuda a crear la respuesta de Azure OpenAI utilizando el modelo de chat de tu elección, habilitando el historial de chat.


In [3]:
def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

In [4]:
def get_completion_from_messages(messages, model=chat_model, temperature=0):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]



#### Prueba las funciones auxiliares

In [5]:
messages =  f"""
tell me a joke.
"""
response = get_chat_completion(messages)
print(response)

Why did the tomato turn red? Because it saw the salad dressing!


In [6]:
messages =  [      
    {'role':'user', 'content':'tell me a joke'},   
    {'role':'assistant', 'content':'Why did the chicken cross the road'},   
    {'role':'user', 'content':'I don\'t know'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side!


#### Tareas para Estudiantes (preguntas abiertas):
1. Crea una función de finalización de texto (completions) para una temperatura muy alta, entre (0,1).
2. Crea una función de finalización de texto (completions) para un valor muy pequeño de max_tokens a fin de obtener respuestas más cortas.
3. Crea funciones de finalización (completions) con 2 valores diferentes de penalización por diversidad entre (0,2).

Prueba las funciones de finalización (completions) que has creado y compara los resultados obtenidos con las pruebas anteriores.

In [7]:
def get_chat_completion(prompt, model=chat_model, temperature=1, max_tokens=200, frequency_penalty=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    temperature=temperature,
    max_tokens = max_tokens,
    frequency_penalty=frequency_penalty,
    top_p = 1.0
    )
    return response.choices[0].message["content"]


def get_completion_from_messages(messages, model=chat_model, temperature=0, max_tokens=200, frequency_penalty=0):
    response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    temperature=temperature,
    max_tokens=max_tokens,
    frequency_penalty=frequency_penalty
    )

    return response.choices[0].message["content"]


In [8]:
# Prueba aquí la función de completación para una temperatura muy alta entre (0,1)
prompt=f"""
Hola, que pasa si el valor de la temperatura de un modelo es muy alta?
"""
temperature_values = [0, 0.2, 0.3, 0.5, 0.7, 1]

for value in temperature_values:
    response = get_chat_completion(prompt, chat_model, value)
    print("-----------------------------------------------------------------------------------")
    print("\n El valor de la temperatura para esta respuesta es:", value, "La respuesta es: \n")
    print(response)


-----------------------------------------------------------------------------------

 El valor de la temperatura para esta respuesta es: 0 La respuesta es: 

Si el valor de la temperatura de un modelo es muy alta, puede indicar que el modelo está sobrecalentado y puede haber problemas en el rendimiento del modelo. Esto puede llevar a una mayor inestabilidad y errores en las predicciones del modelo. Es importante ajustar la temperatura del modelo para asegurarse de que esté en un rango óptimo para obtener resultados precisos y confiables.
-----------------------------------------------------------------------------------

 El valor de la temperatura para esta respuesta es: 0.2 La respuesta es: 

Si el valor de la temperatura de un modelo es muy alta, puede significar que el modelo está sobrecalentado y puede haber problemas en su funcionamiento. Esto puede provocar un aumento en el consumo de energía, un mayor desgaste de los componentes y una disminución en la vida útil del modelo. Ade

In [9]:
# Prueba aquí la función de completación para un valor max_tokens muy bajo para obtener respuestas más cortas
prompt = f"""
Hola! Podrias hacer un resumen del señor de los anillos?
"""
short_response = get_chat_completion(prompt, chat_model, temperature=0.5, max_tokens=20)

print(short_response)

El Señor de los Anillos es una trilogía de novelas escritas por J


In [10]:
# Prueba aquí las funciones de completación con 2 valores diferentes de penalización por diversidad entre (0,2).
prompt = f"""
Hola! Podrias hacer un resumen del señor de los anillos?
"""

response1 = get_chat_completion(prompt, chat_model, temperature=0.5, max_tokens=200, frequency_penalty=0)
response2 = get_chat_completion(prompt, chat_model, temperature=0.5, max_tokens=200, frequency_penalty=2)

print("Esta es la primer respuesta: \n", response1)
print("-----------------------------------------------")
print("Esta es la segunda respuesta: \n", response2)


Esta es la primer respuesta: 
 El Señor de los Anillos es una trilogía de novelas escritas por J.R.R. Tolkien. La historia sigue a un hobbit llamado Frodo Bolsón, quien es elegido para llevar un anillo mágico que puede dar el control al malvado señor oscuro Sauron. Frodo se une a un grupo de amigos, incluyendo un mago llamado Gandalf, un elfo llamado Legolas, un enano llamado Gimli y un humano llamado Aragorn, para llevar el anillo a la destrucción en el Monte del Destino. A lo largo del camino, enfrentan muchos peligros, incluyendo orcos, trolls y otros seres malvados. Al final, Frodo y sus amigos logran destruir el anillo y derrotar a Sauron, salvando la Tierra Media.
-----------------------------------------------
Esta es la segunda respuesta: 
 El Señor de los Anillos es una trilogía de novelas escrita por J.R.R. Tolkien que cuenta la historia del joven hobbit Frodo Bolsón, quien se embarca en una peligrosa misión para destruir el Anillo Único, un poderoso objeto creado por el malv

### 1.2 Ingeniería de Mensajes del Sistema
Los usuarios pueden lograr que los modelos respondan en el tono deseado ajustando el mensaje del sistema.

#### 1.2.1 Cambio de Tono

In [11]:
messages =  [      
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Why was the math book sad?
Because it had too many problems.


In [12]:
messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Why did the noblewoman refuse to give a knight her phone number? Because she preferred to have a trebuchet!


#### Tarea para Estudiantes (pregunta abierta):
Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.


In [13]:
# Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.
messages =  [      
    {'role':'assistant', 'content':'You are an assistant that speaks like Gandalf from The Lord of The Rings'},
    {'role':'user', 'content':'Hola Gandalf, me podrias contar un chiste?'}
]
response = get_completion_from_messages(messages, temperature=0.5)
print(response)

¡Por supuesto, mi querido amigo! ¿Por qué los hobbits siempre están tan contentos? Porque tienen una vida muy corta y no tienen tiempo para preocuparse demasiado. ¡Ja, ja, ja! ¿Te ha gustado el chiste?


#### 1.2.2 Recordar el Nombre de la Empresa

In [14]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},  
{'role':'user', 'content':'Hi, my name is Mel.'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I do not have access to personal information like that. Can you tell me your employer's name and I can help you with more specific details about that company?


#### Tarea para Estudiantes:
Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.

In [15]:
# Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.
messages =  [  
{'role':'system', 'content':'You are friendly chatbot, from the company named Softtek.'},  
{'role':'user', 'content':'Hi, my name is Mel.'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Of course! You work for Softtek, which is the company that I represent as a chatbot. Softtek is a global provider of IT services and business process solutions, with more than 30 years of experience in the industry. How can I assist you with anything else, Mel?


## 2. Principios de Prompting Iterativo
Es una buena práctica realizar un prompting iterativo para que el modelo pueda generar la respuesta más adecuada según las especificaciones del usuario.
- **Principio 1: Escribe instrucciones claras y específicas**
- **Principio 2: Dale tiempo al modelo para "pensar"**



### 2.1 Escribe instrucciones claras y específicas

#### Táctica 1: Delimitadores

#### Tarea para Estudiantes:
Usa delimitadores para indicar claramente las partes distintas de la entrada.

Los delimitadores pueden ser cualquier cosa como: ````, """, < >, `<tag> </tag>`, `:`

In [16]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

In [17]:
# Usa delimitadores para indicar claramente las partes distintas de la entrada y pide al modelo que resuma el texto.
text = f"""
<Query> Podrias resumir el siguiente texto?<Query>
<Intro>En este texto vamos a hablar de ditintos tipos de bases de datos<Intro>
<Desarrollo>
Exite una gran variedad de bases de datos, aunque en general se suelen dividir en dos grupos principales:
Las bases de datos SQL que utilizan un modelo relacional y las bases de datos No-SQL que utilizan otros modelos, como las bases de grafos, key-value o column family.
<Desarrollo>
"""

response = get_chat_completion(text)

print(response)

En el texto se presenta una breve introducción sobre los diferentes tipos de bases de datos existentes, incluyendo las bases de datos SQL que utilizan un modelo relacional y las bases de datos No-SQL que utilizan otros modelos, como las bases de grafos, key-value o column family.


#### Táctica 2: Resumen: especificar el número de palabras, extraer información

#### Texto a resumir

In [18]:
game_review = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

#### Tarea para Estudiantes:
Crea un resumen, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.

In [19]:
# Crea un resumen, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.
messages = [  
{'role':'system', 'content':'You are friendly chatbot, from the company named Softtek.'},  
{'role' : 'user', 'content' : 'Please summarize the following text, focus on the particular special aspects of this olympic games'},
{'role': 'user', 'content':  game_review} ]

summarization = get_completion_from_messages(messages, chat_model, temperature=0.5, max_tokens=150)

print(summarization)

The 2020 Summer Olympics, held in Tokyo, Japan, was an international multi-sport event that took place from 23 July to 8 August 2021. The event was originally scheduled to take place in 2020, but was postponed due to the COVID-19 pandemic. The Games were held largely behind closed doors with no public spectators permitted due to the pandemic, making it the first and only Olympic Games to be held without official spectators. New events were introduced in existing sports, and new sports were added to the Olympic program, including baseball and softball, karate, sport climbing, surfing, and skateboarding. The United States topped the medal count, with China finishing second and host nation Japan finishing third, setting a record for


#### Tarea para Estudiantes:
Intenta "extract" en vez de "summarize" ("extraer" en vez de "resumir")

In [20]:
# Intenta "extract" en vez de "summarize" ("extraer" en vez de "resumir")
messages = [  
{'role':'system', 'content':'You are friendly chatbot, from the company named Softtek.'},  
{'role' : 'user', 'content' : 'Please extract from this text the most important and special aspects of this olympics games'},
{'role': 'user', 'content':  game_review} ]

extraction = get_completion_from_messages(messages, chat_model, temperature=0.5, max_tokens=150)

print(extraction)

The most important and special aspects of the 2020 Summer Olympics are as follows:

1. The event was postponed for the first time in history due to the global COVID-19 pandemic, and it was held without official spectators, making it a unique and historic event.

2. The Games were the most expensive ever, with total spending of over $20 billion.

3. New events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women.

4. New IOC policies allowed the host organizing committee to add new sports to the Olympic program for just one Games,


#### Táctica 3: Inferencia: solicitar emociones, sentimientos o temas

#### Tarea para Estudiantes:
Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña.

Usa un objeto JSON.

In [21]:
review = """
Philip Barker of Inside the Games opined that for many athletes and supporters, \
the tone of the ceremony was dignified and appropriate. Hashimoto stated in a press interview that the flame would "quietly go out", \
which he felt that "It was an apt description of a dignified and low key Ceremony which conveyed a sense of gratitude that the Games had been able to take place at all."\
Dominic Patten of Deadline Hollywood argued that the ceremony was an "uneven mixtape" of contrasts, \
comparing the low-key "celebration of the culture of the Asian power and brow moping acknowledgement of the pandemic" to the jubilant Paris segment, \
as well as clichÃ©-filled speech of Thomas Bach. Alan Tyres of The Daily Telegraph discussed the IOC updated motto as a sign of things to come. \
He stated, "The updated Olympic motto of 'faster, higher, \
stronger â€“ together' fits with how sport is covered and contextualised at this moment in history: \
inclusion, diversity, justice and a duty of care to the athletes must be taken into consideration as much as performance." \
He also discussed the strangeness of the ceremony, as it was performed without a stadium audience.
"""

In [22]:
# Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña. Usa un objeto JSON
messages = [
    {'role': 'system', 'content': 'You are an sentiment and emotions extractor. Your job is to identify emotions and sentimento from a text given by the use. The format of your answer must be the following: <Sentiment> here write the text that evokate this sentiment <Sentiment>, you must writ a list of all your sentiment detected'},
    {'role': 'user', 'content': review}
]

emotions = get_completion_from_messages(messages, chat_model, 0.5, 200, 0)

print(emotions)

Sentiments Detected:
- Positive: "dignified", "appropriate", "gratitude", "low-key"
- Negative: "uneven mixtape", "brow moping acknowledgement"
- Neutral: "strangeness"


#### Desafío para Estudiantes:
Deduce 3 temas (tópicos) de la historia a continuación.

In [23]:
story = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

In [24]:
# Deduce 3 temas (tópicos) de la historia a continuación.

topic_system_prompt = 'You are a topic extractor IA, your job its to extract topics from a text given by the user. You must label all the text of the given text like this: <Topic> followed with the text labeled with the Topic'

messages = [
    {'role': 'system', 'content':topic_system_prompt },
    {'role': 'user', 'content': story}
]

topics = get_completion_from_messages(messages, chat_model, 0.5, 200, 0)

print(topics)


<Olympic Games> The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan.

<Host City> Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.

<Postponement> Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic.

<Expenses> The Games were the most expensive ever, with total spending of over $20 billion.

<No Spectators> It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official


#### Táctica 4: Transformación: especifica el idioma objetivo y el estilo de escritura, y solicita una revisión gramatical.

#### Tarea para Estudiantes:
Traductor Universal

Personas de todo el mundo desean conocer las noticias de los Juegos Olímpicos en su idioma nativo. En este caso, las noticias deben ser traducidas a diferentes idiomas. Traduce cada elemento de noticias a continuación tanto al coreano como al inglés.

In [25]:
news = [
"Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, \
based on the FINA Points System of June 28, 2021.",
"比赛包括两轮：预赛和决赛。预赛成绩最好的8次接力队晋级决赛。必要时使用游泳比赛来打破平局以晋级下一轮。"]

In [26]:
# Escribe aquí el código para la tarea .
translator_system_prompt = 'You are a translator. Your job its to translate the text provided by the user to korean if its in english and to english if the source language its korean, your response must by formated with the original text followed by the translation. For example: \
    user provided text: El dia martes a las 9 de la mañana hubo un choque en la ruta 9 \
    the excpected answer should besomething like: "Translating the following text  El dia martes a las 9 de la mañana hubo un choque en la ruta 9 to korean: <Translation>"'

messages = [
    {'role': 'system', 'content': translator_system_prompt},
    {'role': 'user', 'content': 'Hola, podrias traducir las siguientes noticias al ingles y al coreano respectivamente?'}
]



for new in news: 
    messages.append({'role': 'user', 'content': new})


translations = get_completion_from_messages(messages, chat_model, 0.5, 500, 0)

print(translations)

Translating the following text "Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, based on the FINA Points System of June 28, 2021." to Korean: 
팔레스타인은 2021년 6월 28일 FINA 포인트 시스템을 기반으로 각각의 개인 종목에서 상위 랭킹 수영 선수 1명(성별에 따라)을 올림픽에 파견하는 대학적 초청을 받았습니다.

Translating the following text "The competition consists of two rounds: the heats and the final. The top eight relay teams with the best times in the heats advance to the final. If necessary, swim-offs are used to break ties for advancement to the next round." to English:
The competition consists of two rounds: the heats and the final. The top eight relay teams with the best times in the heats advance to the final. If necessary, swim-offs are used to break ties for advancement to the next round.


#### Tarea para Estudiantes:
Transformación de Tono

La escritura puede variar según la audiencia objetivo. ChatGPT puede producir diferentes tonos. Transforma el siguiente mensaje en una carta de negocios.

In [27]:
original_message = "David, it's John! OMG, the Olympic game is so crazy"

In [28]:
# Escribe aquí el código para la tarea .
business_letter = get_chat_completion('Please, change the tone of the following message to a formal business letter: ' + original_message)

print(business_letter)

Dear David,

I hope this letter finds you well. I wanted to touch base with you regarding the Olympic games and share my excitement. The competition has been nothing short of breathtaking, and I am pleased to have had the opportunity to witness such an impressive display of athleticism.

Thank you for your time, and I look forward to hearing your thoughts on the games.

Sincerely,

John


#### Tarea para Estudiantes:
Conversión de Formato

ChatGPT puede traducir entre formatos. El prompt debe describir los formatos de entrada y salida. Convierte los siguientes datos JSON al formato HTML.

In [29]:
data_json = { "The 2020 Summer Olympics Opening Ceremony audience name list" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}


In [30]:
# Escribe aquí el código para la tarea .

html_response = get_chat_completion("Hola, podrias pasar el siguiente json a formato html?" + str(data_json))

print(html_response)

<table>
  <thead>
    <tr>
      <th>The 2020 Summer Olympics Opening Ceremony audience name list</th>
    </tr>
    <tr>
      <th>Name</th>
      <th>Email</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Shyam</td>
      <td>shyamjaiswal@gmail.com</td>
    </tr>
    <tr>
      <td>Bob</td>
      <td>bob32@gmail.com</td>
    </tr>
    <tr>
      <td>Jai</td>
      <td>jai87@gmail.com</td>
    </tr>
  </tbody>
</table>


#### Tarea para Estudiantes:
Revisión ortográfica y gramatical del siguiente texto.

Para indicarle al modelo LLM que deseas que revise tu texto, instruye al modelo para que 'revise' o 'revise y corrija'.

In [31]:
check_text = [ 
  "Alongside the main Emblem blue, the five other colors use in the branding of the 2020 Games is : Kurenai red,   Ai blue,   Sakula pink,   Fuji purple, and   Matsuba green.",
  "The competition have three round: heats, semifinals, and a final."
]

In [32]:
def add_user_instructions(messages, prompt_list):
    for prompt in prompt_list:
        messages.append({'role': 'user', 'content': prompt})
    return messages

In [33]:
# Escribe aquí el código para la tarea .
grammar_assistance_system_prompt = 'You are a bot that helps checking the grammar, fixing mistakes and suggesting potential improvements. \
    given a text by the user your job its to find all grammar mistakes and fix it. \
    Your answer must start with: "Here you have your text fixed!"'

messages = [
    {'role': 'system', 'content': grammar_assistance_system_prompt}
]

messages = add_user_instructions(messages, check_text)

fixed_text = get_completion_from_messages(messages, chat_model, 0.5, 500, 0)

print(fixed_text)


Here you have your text fixed: 

Alongside the main emblem blue, the five other colors used in the branding of the 2020 Games are Kurenai red, Ai blue, Sakula pink, Fuji purple, and Matsuba green. 

The competition has three rounds: heats, semifinals, and a final.


#### Táctica 5: Ampliación: personaliza la respuesta automática

#### Tarea para Estudiantes:
Personaliza la respuesta automática al siguiente correo electrónico de un cliente.

El cliente tuvo un problema al comprar un boleto para los Juegos Olímpicos.

In [34]:
# given the sentiment from the tactic on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a ticket transaction
review = f"""
I bought the ticket of "Men's 100 metre freestyle swimming" game last week.\
The transaction went through successfully. However, I still have not received the ticket.\
Over one week has passed.\
"""

In [35]:
# Escribe aquí el código para la tarea .
automatic_ticket_support_prompt = 'You are a chatbot that helps the user with its problems using our service for buying tickets. Its important that your answer have to be based on the user sentiment'

messages = [
    {'role': 'system', 'content': automatic_ticket_support_prompt}
]

messages = add_user_instructions(messages, ["user sentiment its: " + sentiment, review])

automatic_response = get_completion_from_messages(messages, chat_model, 0.5, 200, 0)

print(automatic_response)

I'm sorry to hear that you haven't received your ticket yet. I understand that this can be frustrating. Let me check on the status of your ticket for you. Can you please provide me with your order number?


#### Táctica 6: Chatbot: personalizar conversaciones para tareas o comportamientos específicos

#### Tarea para Estudiantes:
Crear una conversación con el chatbot para saber dónde se celebraron los Juegos Olímpicos de Verano de 2020.

In [36]:
# Escribe aquí el código para la tarea .
olympics_system_prompt = "You are a chatbot thar helps the user to get information about the Summer Olympic Games 2020"

messages = [
    {'role': 'system', 'content': olympics_system_prompt},
    {'role': 'user', 'content': 'Hola! Me podrias brindar informacion para saber en donde se celebraron los juegos olimpicos de verano de 2020?'}
]

response = get_completion_from_messages(messages, chat_model, 0.5, 200, 0)

print(response)

¡Hola! Claro que sí, los Juegos Olímpicos de Verano de 2020 se celebraron en la ciudad de Tokio, Japón. ¿Te gustaría saber algo más sobre los juegos olimpicos de verano de 2020?


### 2.2 Dale tiempo al modelo para "pensar"

#### Táctica 1: Especificar los pasos necesarios para completar una tarea
A veces puedes ayudar al modelo a "ralentizarse" y ofrecer respuestas más robustas y detalladas especificando los pasos que debe seguir.

Pidamos la salida en varios formatos especificados.

In [37]:
prompt = f"""
Your task is to help a journalist summarize information from the article for publication.

Write a title based on the information provided in the context delimited by triple backticks. 
The title should be short, catchy, and reflective of the article's narrative.

After the title, generate five keywords from the context.

After the keywords, include a table to organize the information. 
The table should have two columns. In the first column is the title.
In the second column include the keywords as a list.

Give the table the title 'Article Publishing Information'.

Format everything as HTML that can be used in a website.
Place the title in a <div> element.

Context: ```{text}

""" 

get_chat_completion(prompt)

'<div>\n  <h1>Tipos de Bases de Datos</h1>\n</div>\n\n<p>Categorías: Bases de Datos, SQL, No-SQL, Modelos</p>\n\n<table>\n  <tr>\n    <th>Título</th>\n    <th>Palabras Clave</th>\n  </tr>\n  <tr>\n    <td>Dos grupos principales de bases de datos</td>\n    <td>SQL, modelo relacional, No-SQL, bases de grafos, key-value, column family</td>\n  </tr>\n\n</table>\n\n<h2>Información de Publicación del Artículo</h2>'

#### Táctica 2: Instruir al modelo para que desarrolle su propia solución

Existen dos métodos principales que demostraremos en esta sección para lograr que el modelo resuelva un problema en lugar de apresurarse a una conclusión: cadena de pensamiento (chain-of-thought) y encadenamiento (chaining). Estas estrategias pueden llevar a una mayor precisión, detalle y la capacidad de resolver desafíos complejos.


2.1 - Prompting de cadena de pensamiento (Chain-of-thought)
- Pedir al modelo que razone
- Ejemplo de un solo disparo (one-shot)
- Razonamiento del chatbot

2.2 - Encadenamiento (Chaining)

Continuemos trabajando con el conjunto de datos de los Juegos Olímpicos.

#### 2.2.1 Prompting de Cadena de Pensamiento (Chain-of-Thought Prompting)

Hagamos un poco de matemáticas. Los modelos de GPT ocasionalmente no resuelven muy bien problemas matemáticos directos, así que guiemos al modelo a través del problema.

Descomponemos las tareas en piezas más pequeñas.

Lee más sobre los métodos y la investigación en el documento técnico: https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md

#### Táctica 1: Puedes comenzar pidiéndole específicamente al modelo que piense paso a paso.

In [38]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

# From Azure documentation
prompt = "Who was the most decorated (maximum medals) individual athlete in the Olympic games that were held at Sydney? Take a step-by-step approach in your response, cite sources and give reasoning before sharing final answer in the below format: ANSWER is: <name>"
get_chat_completion(prompt)

'Step 1: Identify the year of the Olympic games held in Sydney. The Summer Olympic Games were held in Sydney in the year 2000.\n\nStep 2: Determine the criteria for measuring an athlete\'s "decorations" in the Olympics. The number of medals won by an athlete is the most common way of measuring their success and decorations in the Olympics.\n\nStep 3: Research the list of most decorated athletes in the 2000 Summer Olympics. According to Olympic.org, the most decorated athlete in the 2000 Summer Olympics was Michael Phelps, an American swimmer.\n\nStep 4: Review the number of medals won by Michael Phelps in the 2000 Summer Olympics. Michael Phelps won a total of 6 medals (3 gold, 2 silver, and 1 bronze) at the 2000 Summer Olympics in Sydney.\n\nStep 5: Compare the number of medals won by Michael Phelps to other athletes in the 2000 Summer Olympics. According to Olympic.org, no'

In [39]:
# Another example
prompt = f"""
What is the largest time zone difference is between the top two countries who 
won the most gold medals in the 2020 Tokyo olympics?

Use the context below and think aloud as you solve the problem, step-by-step.

Context: {text}
"""
get_chat_completion(prompt)

"To solve this problem, we need to identify the top two countries who won the most gold medals in the 2020 Tokyo Olympics, and then determine the time zone difference between them. According to the context, the United States topped the medal count by both total golds and total medals, with China finishing second by both respects. Therefore, we need to find the time zone difference between the United States and China. \n\nTo do this, we need to know the time zones of both countries. The United States has multiple time zones, but we can take the Eastern Time Zone as a reference since it is the most populous and the location of many of the country's major cities. China, on the other hand, has a single time zone called China Standard Time (CST), which is 12 hours ahead of Eastern Time. \n\nTherefore, the time zone difference between the United States and China is 12 hours. This means that when it is noon in the Eastern Time Zone of the United States"

#### Táctica 2: Ejemplo de un solo disparo (One-shot)
Otra táctica común es proporcionar un ejemplo de una consulta y una respuesta ideal. El modelo aprenderá de ese ejemplo y aplicará los patrones a una nueva pregunta.

In [40]:
# Notice how this response may not be ideal, or the most accurate.
prompt = f"""
    The United States has 113 total medals, 39 of which are gold medals. 
    
    Great Britain has 64 medals and 22 gold medals. 
    
    How many more silver and bronze medals does the United States have over Great Britain?
"""

get_chat_completion(prompt)

"The United States has 74 medals that are not gold medals (113 total medals - 39 gold medals).\nGreat Britain has 42 medals that are not gold medals (64 total medals - 22 gold medals).\nTherefore, the United States has 74 - 42 = 32 more non-gold medals than Great Britain.\nSince silver and bronze medals are both non-gold medals, we can't determine exactly how many of each the United States has more than Great Britain. However, we do know the difference between their total non-gold medals."

In [41]:
# Give the model a one-shot example to solve the question more thoroughly

prompt = f"""
Question: The United States has 113 total medals, 39 of which are gold medals. How many medals are silver or bronze?
Answer:
[Step 1] - There are three types of medals: gold, silver, and bronze
[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (39) and total (113) counts. 
            113-39=74, so there are 74 silver and bronze medals combined. The answer is 74.

===

Answer the following question using similar steps above.

Question: China has 89 total medals, 38 of which are gold medals. How many silver and bronze medals do they have?
Answer:
"""

get_chat_completion(prompt)

'[Step 1] - There are three types of medals: gold, silver, and bronze\n[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (38) and total (89) counts. \n            89-38=51, so there are 51 silver and bronze medals combined. The answer is 51.'

#### Táctica 3: Razonamiento de cadena de pensamiento del chatbot (chain-of-thought reasoning)
Puedes incorporar lógica utilizando variables para que el chatbot pueda demostrar de manera dinámica formas específicas de pensar sobre un problema.

**Para ingresar la información, aparecerá un cuadro de texto en la parte superior de la pantalla. Deberías ver una ventana emergente. Escribe 'quit' si quieres salir.**

In [42]:
# Ask the bot to help you make a decision such as deciding whether to take a job or choose between restaurants.
# If the model does not immediately respond to your query, wait 1-5 seconds and retype it.
# If it is not allowing you to give an input, restart the kernel in the navigation bar.
# Type "quit" to end the session

context = '''
        You are a decision bot. Your job is to help come to decision by asking a series of questions one at a time and coming to a reasonable decision based on the information provided.

        You will use the following format to help create the series of questions.

        Template: 
        [Problem/Scenario/Question]: [Provide a brief description of the problem, scenario, or question.]

        Chain of thought:

        [Step 1]: Identify the [key element/variable] in the [problem/scenario/question].
        [Step 2]: Understand the [relationship/connection] between [element A] and [element B].
        [Step 3]: [Analyze/Evaluate/Consider] the [context/implication] of the [relationship/connection] between [element A] and [element B].
        [Step 4]: [Conclude/Decide/Determine] the [outcome/solution] based on the [analysis/evaluation/consideration] of [element A], [element B], and their [relationship/connection].
        [Answer/Conclusion/Recommendation]: [Provide a coherent and logical response based on the chain of thought.]

        You will guide the user though a series of questions one at a time. The first question is broad, and they subsequent questions become more specific. 

        Begin by introducing yourself and asking the first question (step 1) only and nothing else, in simple and easy way.
        '''

conversation=[{"role": "system", "content": context}]

while(True):
    print(len(conversation))
    if len(conversation) == 1:
        response = get_completion_from_messages(conversation)
        conversation.append({"role": "assistant", "content": response})
        print("\n" + response + "\n")
    
    user_input = input('Enter your response: ')
    if user_input.lower() == "quit":
        break 
    conversation.append({"role": "user", "content": user_input})
    
    response = get_completion_from_messages(conversation)
    conversation.append({"role": "assistant", "content": response})
    print("\n" + response + "\n")


1

Hello! I am a decision bot. What is the problem, scenario, or question you need help with?



#### 2.2.2 - Encadenamiento / Chaining
Similar a algunos ejemplos anteriores, puedes usar las salidas del modelo de consultas anteriores en otras consultas. Más adelante en el Hack te mostraremos cómo hacer esto a escala.

In [43]:
# Extract medal counts for each country from the news article
# Write an article about the summarized information
# Provide a title for the summary

text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

prompt = f"""
    Based on the context below, write a JSON object that contains the number of gold and total medals for each country.
    Context: {text}
"""
num_medals_dict = get_chat_completion(prompt)

prompt = f"""
    Write a brief article about the winners and losers of the Olympics based on medal count:
    {num_medals_dict}
"""
summary = get_chat_completion(prompt)
print(summary)

prompt = f"""
    Give the summary a title in 5 words:
    {summary}
"""
title = get_chat_completion(prompt)
print(title)

The Tokyo 2020 Olympics culminated on August 8, 2021, with the United States emerging as the clear winner of the medal count. The US athletes won a total of 113 medals, including 39 golds, 41 silvers, and 33 bronzes. China came in second place with a total of 89 medals, including 38 golds, 32 silvers, and 18 bronzes. These two countries were the clear winners, with the United States winning nearly 25% more medals than China.

The third place went to Japan with a total of 58 medals, including 27 golds, 14 silvers, and 17 bronzes. The host country had a strong showing in many sports, including judo, swimming, and gymnastics.

Great Britain came in fourth place with a total of 64 medals, including 22 golds, 21 silvers, and 21 bronzes. Team GB
United States Dominates Tokyo Olympics


#### Tarea para Estudiantes (Encadenamiento / Chaining):

Tu tarea ahora es escribir código que determine el país que ganó la mayor cantidad de medallas de plata y bronce combinadas.

Podemos ver que el modelo tiene un rendimiento deficiente al responder la pregunta directamente.

In [44]:
prompt = f"""
    Based on the context, which country had the most silver and bronze medals?
    Context: {text}
"""

get_chat_completion(prompt)

'The context does not provide clear information about which country had the most silver and bronze medals.'

In [50]:
print(num_medals_dict)

{
  "United States": {
    "gold": 39,
    "total": 113
  },
  "China": {
    "gold": 38,
    "total": 89
  },
  "Japan": {
    "gold": 27,
    "total": 58
  },
  "Great Britain": {
    "gold": 22,
    "total": 64
  },
  "ROC": {
    "gold": 20,
    "total": 71
  },
  "Bermuda": {
    "gold": 1,
    "total": 1
  },
  "Philippines": {
    "gold": 1,
    "total": 1
  },
  "Qatar": {
    "gold": 1,
    "total": 2
  },
  "Burkina Faso": {
    "gold": 0,
    "total": 1
  },



Escribe prompts en la celda a continuación que ayudarán al modelo a responder la pregunta desglosando las tareas en diferentes pasos. Asegúrate de que responda solo con la información que se le proporcionó. Este concepto de fundamentación se presentará con más detalle en el Reto 3.

Deberías poder lograr que el modelo responda la pregunta en 2-3 pasos.

In [ ]:
# Escribe aquí el código para la tarea .

text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

prompt = f"""
    Based on the context below, write a JSON object that contains the number of gold and total medals for each country.
    Context: {text}
"""

num_medals_dict = get_chat_completion(prompt)

#print(num_medals_dict)

context = f"""
    You are a chat bot wich only job is to answer, based on the context below, wich Nation won more Silver and Bronze medal combined.
    Use this data to ground your answer: {num_medals_dict}.

    Answer:
    
    [Step 1] - There are three types of medals: gold, silver, and bronze
    [Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (39) and total (113) counts. 
            113-39=74, so there are 74 silver and bronze medals combined. The answer is 74.

===

Answer the user question using similar steps above.

"""

user_question = {'role': 'user', 'content': 'Quien gano mas medaalas de bronce y plata combinadas en los juegos olimpicos de tokio 2020?'}


conversation=[{"role": "system", "content": context}, user_question]

response = get_completion_from_messages(conversation, chat_model, 0.5)

print(response)


{
    "United States": {
        "gold": 39,
        "total medals": 113
    },
    "China": {
        "gold": 38,
        "total medals": 89
    },
    "Japan": {
        "gold": 27,
        "total medals": 58
    },
    "Great Britain": {
        "gold": 22,
        "total medals": 64
    },
    "ROC": {
        "gold": 20,
        "total medals": 71
    },
    "Bermuda": {
        "gold": 1,
        "total medals": 1
    },
    "Philippines": {
        "gold": 1,
        "total medals": 1
    },
    "Qatar": {
        "gold": 1,
        "total medals": 2
    },
    "Burkina Faso": {
        "gold": 0,
       
Según los datos oficiales, el país que ganó más medallas de plata y bronce combinadas en los Juegos Olímpicos de Tokio 2020 fue Estados Unidos, con un total de 74 medallas de plata y bronce combinadas. China quedó en segundo lugar con un total de 51 medallas de plata y bronce combinadas.


## Conclusión

En este primer desafío, cubrimos cómo interactuar con Azure OpenAI para diferentes objetivos. Esperamos que hayas podido ver la versatilidad dinámica de los modelos y cómo pueden usarse para resolver una variedad de problemas utilizando diferentes técnicas.

Le dimos a la API piezas cortas de texto usando variables fijas. En el próximo conjunto de desafíos, verás cómo usar la API con conjuntos de datos más grandes.